# Model Testing


## Setup


In [1]:
from scripts.data_loader import load_data, split_features_targets
from scripts.preprocessing import preprocess_data, create_features
from scripts.model_testing import test_independant_models, find_top_models

from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    RANSACRegressor,
    TheilSenRegressor,
    HuberRegressor,
)
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    ExtraTreesRegressor,
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import BaggingRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [2]:
df = load_data("data")
df = preprocess_data(df)
# df = create_features(df)

features, targets = split_features_targets(df)

## Initial Testing


### Models


In [3]:
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1),
    AdaBoostRegressor(n_estimators=50, learning_rate=0.1),
    ExtraTreesRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    DecisionTreeRegressor(max_depth=10),
    KNeighborsRegressor(n_neighbors=5, n_jobs=-1),
    MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500),
    SVR(kernel="linear", C=1.0, epsilon=0.1),
    LinearSVR(C=1.0, epsilon=0.1, max_iter=1000, tol=1e-3),
    NuSVR(kernel="linear", C=1.0, nu=0.5),
    GaussianProcessRegressor(n_restarts_optimizer=0, normalize_y=True),
    PLSRegression(n_components=6),
    KernelRidge(alpha=1.0, kernel="linear"),
    TheilSenRegressor(max_subpopulation=1e4, n_subsamples=None),
    HuberRegressor(epsilon=1.35, max_iter=1000),
    BaggingRegressor(n_estimators=10, max_samples=0.5, n_jobs=-1),
    VotingRegressor(
        estimators=[
            ("lr", LinearRegression()),
            ("rf", RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1)),
            (
                "gbr",
                GradientBoostingRegressor(
                    n_estimators=50, max_depth=3, learning_rate=0.1
                ),
            ),
        ]
    ),
    StackingRegressor(
        estimators=[
            ("lr", LinearRegression()),
            ("rf", RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1)),
            (
                "gbr",
                GradientBoostingRegressor(
                    n_estimators=50, max_depth=3, learning_rate=0.1
                ),
            ),
        ]
    ),
    XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1),
    LGBMRegressor(
        n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1, verbose=-1
    ),
]

### Results


In [4]:
# Ice Thickness
find_top_models(test_independant_models(models, features, targets["ice_thickness"])).head()

C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,MSE,MAE,R2
RandomForestRegressor,0.001777,0.019700,0.972237
BaggingRegressor,0.002025,0.020807,0.968360
XGBRegressor,0.002279,0.023679,0.964381
LGBMRegressor,0.002290,0.023875,0.964212
GradientBoostingRegressor,0.002438,0.024580,0.961897


In [5]:
# Ice Velocity
find_top_models(test_independant_models(models, features, targets["ice_velocity"])).head()

C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,MSE,MAE,R2
XGBRegressor,708.512070,6.082729,0.429612
GradientBoostingRegressor,722.024729,6.322429,0.418733
ExtraTreesRegressor,724.382718,6.291751,0.416835
RandomForestRegressor,754.034420,6.230516,0.392964
LGBMRegressor,837.682476,7.165042,0.325623


In [6]:
# Ice Mask
find_top_models(test_independant_models(models, features, targets["ice_mask"])).head()

C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,MSE,MAE,R2
XGBRegressor,0.043564,0.067745,0.953909
RandomForestRegressor,0.044779,0.057812,0.952624
GradientBoostingRegressor,0.046172,0.070373,0.951149
LGBMRegressor,0.048738,0.076469,0.948435
BaggingRegressor,0.049897,0.061482,0.947209


## Further Testing


### Models


In [7]:
MAX_DEPTH = 50
N_ESTIMATORS = 150
N_JOBS = -1

models = [
    RandomForestRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, n_jobs=N_JOBS),
    GradientBoostingRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=0.1),
    ExtraTreesRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, n_jobs=N_JOBS),
    BaggingRegressor(n_estimators=N_ESTIMATORS, max_samples=0.5, n_jobs=N_JOBS),
    XGBRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=0.1, n_jobs=N_JOBS),
    LGBMRegressor(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=0.1, n_jobs=N_JOBS, verbose=-1
    ),
]

### Results


In [9]:
results = test_independant_models(models, features, targets["ice_velocity"], test_split=0.1, subset=False)
results.head(6)

KeyboardInterrupt: 